In [1]:
# Requirements
# Python 3
# Python libraries: pandas

# Can be downloaded from python.org
# pandas can be downloaded by running 'pip install pandas' from the terminal 
# (may need to use pip3 instead of pip)


# How To Use

# Download zip and unzip it. 
# Run setup script: hit shift and right-click in the unzipped folder and click on 'Open PowerShell window here'.
# Type `python fgstSetup.py` in the terminal and wait for it to complete. 
# Run data collection script in PowerShell after each session by running `python fallGuysData.py` 
# (May need to use python3 instead of python)

# Note: 
# If you forget to run for a session, run 'python fallGuysData.py Player-prev.log' 
# before running python fallGuysData.py for your new session
# word that better

In [2]:
# Things to do...

# get script to run when Player.log is updated...or just run after each session
# fix variable naming convention to use _s
# new repo for it

In [3]:
import os, datetime, time
import pandas as pd

# https://stackoverflow.com/a/10854983
offset = time.timezone if (time.localtime().tm_isdst == 0) else time.altzone
tz = offset / 60 / 60 
HOURS_DIFFERENTIAL = int(tz) # set time zone each time

# need hex-a-gone and royal fumble
list_of_finals = ['round_fall_mountain', 'round_floor_fall', 'round_royalfumble', 'round_jump_showdown', 
                  'round_fall_mountain_hub_complete']

with open('totalshows.txt') as f:
    total_shows = f.read()
total_shows = int(total_shows.strip())

with open(os.path.join('data', 'session.txt')) as f:
    session_num = f.read()
session_num = int(session_num)

In [5]:
# Notes

# find player id for round - don't think we can do anything about that
# use that as well instead of username
# only affects userEndRoundLines

# check elim time for jump_showdown - seems as though personal end times are only for...
# ...finishing races early

# playing around Daylight Savings switch may result in bad times for that round

# figure out if there's a max number of shows that will be stored in the .log : seems the whole session

In [6]:
def cleanLines(lines):
    return [line.replace('[', '').replace(']', '').replace('>', '').strip() for line in lines]

# splits show into show data and rounds
def roundSplit(lines):
    lines = cleanLines(lines)
    splits = []
    # start of each round in highlights section
    for i, line in enumerate(lines):
        if 'Round' in line:
            splits.append(i)
    
    splits.append(0)
    # split it into rounds
    rounds = [lines[splits[i-1]:splits[i]] if i != len(splits)-1 else lines[splits[i-1]:]
              for i in range(1, len(splits))]
    
    return lines[1:splits[0]], rounds

# takes registered and connected lines
# returns start times for rounds
# (time at which game is found)
def getStartTimes(reg, conne):
    startTimes = []
    
    for i in range(len(conne)):
        registeredTime = datetime.datetime.strptime(reg[i].split('at: ')[-1], '%m/%d/%Y %H:%M:%S %p')
        connectedTime = datetime.datetime.strptime(conne[i].split(': [')[0], '%H:%M:%S.%f')
        
        d = (connectedTime - registeredTime)
        startTimes.append(registeredTime + (d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL)))
    
    return startTimes

def getTimeTaken(start, end):
    d = datetime.datetime.strptime(end, '%H:%M:%S.%f') - start
    # start time was already adjusted for HOURS_DIFFERENTIAL, so need to again
    return str(d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL))[2:] # so hours isn't included

# edit this later change
# grab the last Seasonthing in txt file - countdown in .log
# if its larger than the last one, then the season reset
def getSeason():
    return 2

# gets lines for CompletedEpisode section of a show
def getShowLines(lines, marker):
    finalLines = []
    tempLines = lines[marker:]
    for line in tempLines:
        if line == '':
            continue
        if '>' not in line and '[Round' not in line and '[Complet' not in line:
            break
        if line != "":
            finalLines.append(line)
    return finalLines

def getExtraRoundInfoLines(possLines): # rename?
    rnds = []
    currRnd = possLines[0][1].split()[0]
    currSID = possLines[0][0]
    prevLine = possLines[0][2]
    
    # get last line before map switches (and server ID)
    for i, (serverID, line, line_num) in enumerate(possLines):
        rnd = line.split()[0]
        
        if rnd != currRnd: 
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])
            if (prevLine + 1) == line_num: # sometimes server changes map due to a dropout
                rnds.pop()      
        elif serverID != currSID:
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])
            if (prevLine + 1) == line_num:
                rnds.pop()
        prevLine = line_num
    
    rnds.append(possLines[-1][1])      
    return rnds


# preprocessGrade1 has been retired
# remove lines in-between completed games
def preprocessGrade2(lines):
    print(len(lines))
    lines2 = []
    allGood = False
    badgeID = False
    for line in lines:
        if '[CATAPULT] Login Succeeded' in line:
            allGood = True
        if 'BadgeId:' in line:
            badgeID = True
        if badgeID:
            if '[ClientGlobalGameState] ShutdownNetworkManager' in line or '[ClientGlobalGameState] sending graceful disconnect message' in line:
                badgeID = False
                allGood = False

        if allGood:
            lines2.append(line)
    lines = lines2
    print(len(lines))
    return lines

# changes to the data extraction removed the need for this
def preprocessGrade3(lines):
    #print(len(lines))
    start_conn = -1
    to_remove = []
    in_conn = False
    for i, line in enumerate(lines):
        if "[StateConnectToGame] We're connected to the server!" in line:
            start_conn = i
            in_conn = True
        if 'reports that it is not yet ready to accept connections.' in line:
            if in_conn:
                to_remove.append([start_conn, i])
                in_conn = False
    
    temp_lines = []
    for i, line in enumerate(lines):
        for check in to_remove:
            if i >= check[0] and i <= check[1]:
                continue
            temp_lines.append(line)
            
    #print(len(temp_lines))
    return temp_lines
    
# remove spectated rounds
def preprocessGrade4(lines):
    #print(len(lines))
    start_round = -1
    to_remove = []
    in_spec = False
    for i, line in enumerate(lines):
        if "Received instruction that server is ending a round, and to rejoin" in line:
            start_round = i
        if 'permission=Spectator' in line:
            in_spec = True
        if '[ClientGameManager] Shutdown' in line:
            if in_spec:
                to_remove.append([start_round, i])
                in_spec = False
                
    # remove selected lines
    temp_lines = []
    for i, line in enumerate(lines):
        to_append = True
        for check in to_remove:
            if i >= check[0] and i <= check[1]:
                to_append = False
        if to_append:
            temp_lines.append(line)
    
    #print(len(temp_lines))
    return temp_lines

# helper function (to account for if round wraps around midnight)
def subtractHours(a, b):
    c = (datetime.datetime.strptime(b, '%H:%M:%S.%f') - datetime.datetime.strptime(a, '%H:%M:%S.%f'))
    return str(c - datetime.timedelta(days=c.days))

# save data in csvs
def saveData(show, roundsList):
    new_shows_df = pd.DataFrame(pd.Series(show)).T
    new_rounds_df = pd.DataFrame(roundsList)
    try:
        #load them
        shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
        rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
        
        if str(show['Start Time'])[:21] in [tm[:21] for tm in shows_df['Start Time'].tolist()]:
            return False
        
        # append
        shows_df = shows_df.append(new_shows_df, ignore_index=True)
        rounds_df = rounds_df.append(new_rounds_df, ignore_index=True)
    except: # first time
        shows_df = new_shows_df
        rounds_df = new_rounds_df
        print('first time...creating csvs')
    
    # write them to their respective files
    shows_df.to_csv(os.path.join('data', 'shows.csv'), index=False)
    rounds_df.to_csv(os.path.join('data', 'rounds.csv'), index=False)
    return True

In [7]:
with open("log_path.txt") as f:
    log_path = f.read()

with open(log_path) as f:
    lines = f.read()

#with open(os.path.join('C:\\Users','Joseph','Downloads','Player.log')) as f:
 #   lines = f.read()

lines = lines.split('\n')
lines = preprocessGrade2(lines)
lines = preprocessGrade4(lines)

23947
23297
23297
22182


In [8]:
# get first line of each new show and usernames used for show
prevUser = '!!!!!!!!!!!!!!!'
lookUser = True
inRound = False
inARound = False
gameMode = 'main_show'

episodeMarkers = []
usernames = []
partySizes = []

# times
reg = []
conne = []
startRoundLines = []
userEndRoundLines = []
actualEndRoundLines = []

possLines = []
gameModes = []
# to find the actual number of players that qualified (for racing rounds)
prevNumLine = ""
numLines = []

# **********************************************************
# go through lines, looking for certain things**************
# **********************************************************
for i, line in enumerate(lines):
    # for username
    if lookUser and 'Sending login request' in line:
        usernames.append(line.split(' player ')[-1].split(' networkID')[0].replace(',', ''))
        prevUser = usernames[-1]
        lookUser = False
    # for type of show (main or alternate) (also called playlist)
    elif 'Chosen Show:' in line: # appears before entering matchmaking solo/group
        gameMode = line.split(':')[-1]
    # signifies start of looking for new episode
    # get size of party
    elif 'Party Size' in line or 'Begin matchmaking solo' in line:
        if 'Begin matchmaking solo' in line:
            partySize = 1
        else:
            partySize = int(line.split(' ')[-1].strip())
    # for show start time
    elif '[QosManager] Registered' in line or 'QosManager: Registered' in line: # for registered time (date)
        to_add_reg = line
    elif "[StateConnectToGame] We're connected to the server!" in line: # for connection time
        to_add_conne = line
    # for server ID and map lines
    elif 'Received NetworkGameOptions from ' in line: 
        tmp = line.split('roundID=')[-1]
        serverID = line.split(' ')[4]
        if 'Default' not in tmp:
            possLines.append([serverID, tmp, i])
    # for start round times and players that qualified from previous round
    elif 'state from Countdown to Playing' in line:
        startRoundLines.append(line.split(': [')[0])
        inRound = True
        inARound = True
        # append last # players achieving obj when hit new round
        if prevNumLine != "":
            numLines.append(prevNumLine.split('=')[-1])
            prevNumLine = ""
    elif '[ClientGameSession] NumPlayersAchievingObjective=' in line: # for total number of players that quality
        prevNumLine = line
    # for end round / player active in round times
    elif '[ClientGameManager] Handling unspawn for player FallGuy' in line and prevUser in line:
        if inRound:
            userEndRoundLines.append(line.split(': [')[0])
            inRound = False
    elif 'Changing local player state to: SpectatingEliminated' in line:
        if inRound:
            userEndRoundLines.append(line.split(': C')[0])
            inRound = False
        if inARound:
            actualEndRoundLines.append(line.split(': C')[0])
            inARound = False
    elif 'Changing state from Playing to GameOver' in line: # 'Changing state from GameOver to Results'
        if inARound:
            inARound = False
            actualEndRoundLines.append(line.split(': [')[0])
    # overall show data
    elif '[CompletedEpisodeDto]' in line: # marker for a good show; only append show stats here
        partySizes.append(partySize)
        gameModes.append(gameMode)
        episodeMarkers.append(i)
        reg.append(to_add_reg)
        conne.append(to_add_conne)
        lookUser = True
        if inARound:
            actualEndRoundLines.append('left')
            inARound = False

# append last # achieving obj
numLines.append(prevNumLine.split('=')[-1])

# if no episodes found, end
if len(episodeMarkers) == 0:
    print('no episodes found') # change
        
# **********************************************************        
# get time user spent in each round ************************        
# (time round starts until they either finish or are eliminated) (just qualify I think)
# **********************************************************
roundTimes = []
for a, b in zip(startRoundLines, userEndRoundLines):
    roundTimes.append(subtractHours(a, b))

# get total round time
actualRoundTimes = []
for a, b in zip(startRoundLines, actualEndRoundLines):
    try:
        actualRoundTimes.append(subtractHours(a, b))
    except ValueError:
        actualRoundTimes.append('uncertain')

# gets start times for each show
startTimes = getStartTimes(reg, conne)


# **********************************************************
# for each show/episode ************************************
# **********************************************************
roundIdx = 0
showsSaved = 0
showsSkipped = 0

rnds = getExtraRoundInfoLines(possLines)
    
for showIdx, (j, user) in enumerate(zip(episodeMarkers, usernames)):
    this_show = total_shows
    total_shows += 1
    
    # get lines for this show
    final_lines = getShowLines(lines, j)
    
    # split data
    showData, rounds = roundSplit(final_lines)
    
    # set show data
    show_dict = {}
    show_dict['Show ID'] = this_show # id
    show_dict['Start Time'] = startTimes[showIdx]
    show_dict['Season'] = getSeason() 
    show_dict['Time Taken'] = getTimeTaken(startTimes[showIdx], final_lines[0].split(': ==')[0]) # approximate time taken
    show_dict['Game Mode'] = gameModes[showIdx]
    show_dict['Final'] = rounds[-1][0].split('|')[-1].strip() in list_of_finals # final
    show_dict['Rounds'] = len(rounds) # num rounds
    show_dict['Username'] = user
    show_dict['Party Size'] = partySizes[showIdx]
    show_dict['addID'] = final_lines[0].split(': ==')[0] # end time
    
    # add other show data
    for line in showData:
        show_dict[line.split(':')[0]] = line.split(':')[1].strip()
    
    # ********************************************
    # get data for each round in show ************
    # ********************************************
    rounds_list = []
    # for each round in the show/episode
    for round_ in rounds: # for list in 2D list
        round_dict = {'Show ID': this_show, 
                      'Round Num': round_[0].split(' ')[1].strip(), 
                      'Map': round_[0].split('|')[1].strip()}
        round_dict['Time Spent'] = roundTimes[roundIdx]
        round_dict['Round Length'] = actualRoundTimes[roundIdx]
        
        # add rest of data from list
        for line in round_[1:]:
            round_dict[line.split(':')[0]] = line.split(':')[1].strip()
        
        # add extra information
        rnd = rnds[roundIdx]
        splts = rnd.split()
        round_dict['Participants'] = splts[4].split('=')[-1] # num people
        round_dict['Qualification Percent'] = splts[7].split('=')[-1].replace(',', '') # qual %
        round_dict['Actual Num Qual'] = numLines[roundIdx]
        
        roundIdx += 1
        rounds_list.append(round_dict)
    
    # ********************************************
    # save ***************************************
    # ******************************************** 
    # save show_dict to one table
    # save each dict in rounds_list to another table
    if not saveData(show_dict, rounds_list):
        showsSkipped += 1
        total_shows -= 1
    else:
        showsSaved += 1

print('csvs saved successfully with {} new shows while skipping {} shows that were already saved'.format(showsSaved, showsSkipped))

with open('totalshows.txt', 'w') as f:
    f.write(str(total_shows))

# save processed lines
with open(os.path.join('data', 'archive', 'session{}.txt'.format(session_num)), 'w') as f:
    f.write("\n".join(lines))
session_num += 1

with open(os.path.join('data', 'session.txt'), 'w') as f:
    f.write(str(session_num))

first time...creating csvs
csvs saved successfully with 3 new shows while skipping 0 shows that were already saved


In [9]:
for line in lines:
    if 'Party Size' in line or 'Begin matchmaking solo' in line:
        print(line)
    if '[Round' in line:
        print(line)
    if '[CompletedEpisodeDto]' in line:
        print(line)

22:15:19.370: [StateMatchmaking] Begin matchmaking solo
22:30:54.018: == [CompletedEpisodeDto] ==
[Round 0 | round_see_saw]
[Round 1 | round_biggestfan]
[Round 2 | round_egg_grab]
[Round 3 | round_wall_guys]
[Round 4 | round_floor_fall]
22:31:08.618: [StateMatchmaking] Begin matchmaking solo
22:40:44.943: == [CompletedEpisodeDto] ==
[Round 0 | round_biggestfan]
[Round 1 | round_tunnel]
[Round 2 | round_hoops]
22:44:07.800: [StateMatchmaking] Begin matchmaking solo
22:59:44.063: == [CompletedEpisodeDto] ==
[Round 0 | round_door_dash]
[Round 1 | round_biggestfan]
[Round 2 | round_hoops_blockade_solo]
[Round 3 | round_fall_ball_60_players]
[Round 4 | round_jump_showdown]


In [10]:
shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
shows_df

,Show ID,Start Time,Season,Time Taken,Game Mode,Final,Rounds,Username,Party Size,addID,Kudos,Fame,Crowns
0,541,2020-11-24 16:15:32.351,2,15:21.667000,main_show,True,5,Fast Swimming Panther,1,22:30:54.018,475,213,0
1,542,2020-11-24 16:31:39.183000,2,09:05.760000,main_show,False,3,Fast Swimming Panther,1,22:40:44.943,215,98,0
2,543,2020-11-24 16:44:39.429000,2,15:04.634000,main_show,True,5,Fast Swimming Panther,1,22:59:44.063,440,196,0


In [11]:
rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
rounds_df

,Show ID,Round Num,Map,Time Spent,Round Length,Qualified,Position,Kudos,Fame,Bonus Tier,Bonus Kudos,Bonus Fame,BadgeId,Participants,Qualification Percent,Actual Num Qual,Team Score
0,541,0,round_see_saw,0:01:12.036000,0:02:06.298000,True,5,30,15,1.0,70,35,silver,59,73,43,NaN
1,541,1,round_biggestfan,0:01:21.969000,0:03:30.020000,True,3,20,10,1.0,70,35,silver,43,73,26,NaN
2,541,2,round_egg_grab,0:02:06.248000,0:01:59.992000,True,10,60,20,0.0,70,35,gold,24,100,0,23.0
3,541,3,round_wall_guys,0:01:25.085000,0:01:28.635000,True,7,50,20,2.0,35,18,bronze,16,60,9,NaN
4,541,4,round_floor_fall,0:01:48.504000,uncertain,False,7,70,25,NaN,0,0,NaN,9,0,0,NaN
5,542,0,round_biggestfan,0:01:08.770000,0:03:30.028000,True,5,30,15,1.0,70,35,silver,60,73,35,NaN
6,542,1,round_tunnel,0:00:57.387000,0:00:57.393000,True,15,20,10,0.0,35,18,gold,35,61,21,NaN
7,542,2,round_hoops,0:02:04.087000,0:02:00.055000,False,18,60,20,NaN,0,0,NaN,21,100,0,48.0
8,543,0,round_door_dash,0:00:43.953000,0:00:49.553000,True,2,30,15,1.0,70,35,silver,60,73,43,NaN
9,543,1,round_biggestfan,0:01:17.919000,0:03:30.039000,True,3,20,10,1.0,70,35,silver,42,73,27,NaN


In [12]:
# https://fallguysultimateknockout.gamepedia.com/Rounds

roundCodeDict = {
    'round_tip_toe': 'Tip Toe',
    'round_block_party': 'Block Party',
    'round_tunnel': 'Roll Out', 
    'round_egg_grab_02': 'Egg Siege', 
    'round_dodge_fall': 'Fruit Chute',
    'round_jump_showdown': 'Jump Showdown',
    'round_door_dash': 'Door Dash',
    'round_jump_club': 'Jump Club',
    'round_biggestfan': 'Big Fans',
    'round_wall_guys': 'Wall Guys',
    'round_match_fall': 'Perfect Match',
    'round_lava': 'Slime Climb',
    'round_gauntlet_03': 'The Whirlygig',
    'round_egg_grab': 'Egg Scramble',
    'round_see_saw': 'See Saw',
    'round_ballhogs': 'Hoarders', 
    'round_gauntlet_01': 'Hit Parade',
    'round_rocknroll': "Rock 'N' Roll",
    'round_fall_mountain_hub_complete': 'Fall Mountain',
    'round_gauntlet_04': 'Knight Fever',
    'round_jinxed': 'Jinxed',
    'round_chompchomp': 'Gate Crash',
    'round_conveyor_arena': 'Team Tail Tag', 
    'round_floor_fall': 'Hex-A-Gone',
    'round_tail_tag': 'Tail Tag',
    'round_hoops': 'Hoopsie Daisy',
    'round_hoops_blockade_solo': 'Hoopsie Legends', 
    # Dizzy Heights
    'round_fall_ball_60_players': 'Fall Ball',
    # Royal Fumble
}

showTypeDict = {
    'main_show': 'Main Show',
    'event_only_survival_3010_to_0511': 'Slime Survivors',
    'event_only_season_2_variation_1011_to_1611': 'Medieval Mix Up',
    'event_only_hard_mode_2111_to_2711': 'Hard Mode',
    
}